In [35]:
import pandas as pd

data = pd.read_parquet("../data/dataset/shs100k2/data.parquet")
data["performer_single"] = data.performer.apply(lambda x: x[0] if len(x) > 0 else '')
cols = ['set_id', 'yt_id', 'title', 'performer', 'performer_single', 'viewcount', 'split', 'TEXT', 'IOB']
data = data[cols]

# get and preprocess wikidata
cols = ["link", "name", "sitelinks"]
data_groups = pd.read_json("../data/wikidata/groups.json")
data_groups.musicalGroupLabel = data_groups.musicalGroupLabel.str.lower()
data_groups.columns = cols
data_groups["type"] = "Group"

#data_humans1 = pd.read_json("../data/wikidata/humans1.json")
#data_humans1.personLabel = data_humans1.personLabel.str.lower()
#data_humans1.columns = cols
#data_humans1["type"] = "Person"
data_humans = pd.read_json("../data/wikidata/soloartists.jsonl")

data_humans["name"] = data_humans.labels.apply(lambda x: x.get("en").get("value").lower() if x.get("en") else None)
data_humans["sitelinks"] = data_humans.sitelinks.apply(len)
data_humans["link"] = data_humans.id.apply(lambda x: "http://www.wikidata.org/entity/" + x)
data_humans = data_humans[["link", "name", "sitelinks"]]
data_humans["type"] = "Person"

data_wikidata = pd.concat([data_groups, data_humans], axis=0, ignore_index=True).drop_duplicates(subset="link")
data_wikidata


,link,name,sitelinks,type
0,http://www.wikidata.org/entity/Q11036,the rolling stones,270,Group
1,http://www.wikidata.org/entity/Q1299,the beatles,237,Group
2,http://www.wikidata.org/entity/Q2331,led zeppelin,234,Group
3,http://www.wikidata.org/entity/Q11649,nirvana,226,Group
4,http://www.wikidata.org/entity/Q45188,coldplay,202,Group
...,...,...,...,...
22273,http://www.wikidata.org/entity/Q18638042,maria roe vincent,3,Person
22274,http://www.wikidata.org/entity/Q18638742,carolina lama,1,Person
22275,http://www.wikidata.org/entity/Q18640609,vera teplitskaya,1,Person
22276,http://www.wikidata.org/entity/Q18640797,irma vila,6,Person


In [36]:
data = pd.merge(data, data_wikidata, how="left", left_on="performer_single", right_on="name")


In [38]:
data.loc[data.name.isna(),"performer_single"].drop_duplicates().sample(60).head(30)


1                    matt monro
2        the hollyridge strings
3                   cilla black
4                 sarah vaughan
8                    brenda lee
                  ...          
78279              ron sexsmith
78280       the sachal ensemble
78281              gary stewart
78282            rodney crowell
78284                    brooks
Name: performer_single, Length: 64501, dtype: object